In [5]:
from mxnet import gluon
print(gluon.__version__)
print(mx.__version__)

/home/idrone2/.local/lib/python3.10/site-packages/mxnet/numpy/utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  bool = onp.bool


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [2]:
import os
import mxnet as mx
from mxnet import gluon
from gluoncv.data import VOCDetection
from gluoncv.data.transforms import presets
from gluoncv.utils import viz

# Define the path to your data.yaml file
data_yaml_path = '/home/idrone2/Desktop/Tea_leaf_disease/data.yaml'

# Load the dataset
class CustomDataset(VOCDetection):
    def __init__(self, root, splits, transform=None, index_map=None, preload_label=True):
        super(CustomDataset, self).__init__(root, splits, transform, index_map, preload_label)

# Load the dataset
dataset = CustomDataset(root=os.path.dirname(data_yaml_path), splits=[('train', 'val')])

# Define the classes
classes = dataset.classes  # This should be automatically loaded from data.yaml

/home/idrone2/.local/lib/python3.10/site-packages/mxnet/numpy/utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  bool = onp.bool


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
from gluoncv.data.transforms import presets

# Define the transformations
train_transform = presets.yolo.YOLO3DefaultTrainTransform(416, 416)
val_transform = presets.yolo.YOLO3DefaultValTransform(416, 416)

# Apply the transformations to the dataset
train_dataset = dataset.transform(train_transform)
val_dataset = dataset.transform(val_transform)

In [ ]:
batch_size = 16

train_loader = gluon.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, last_batch='rollover',
    num_workers=4)

val_loader = gluon.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, last_batch='keep',
    num_workers=4)

In [ ]:
from gluoncv import model_zoo

# Load a pre-trained YOLO model
net = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True)

# Reset the classes to match your dataset
net.reset_class(classes)

In [ ]:
from mxnet import gluon, autograd
from mxnet.gluon import Trainer
from gluoncv.loss import YOLOV3Loss

# Define the loss function
loss_fn = YOLOV3Loss()

# Define the trainer
trainer = Trainer(net.collect_params(), 'adam', {'learning_rate': 0.001})

In [ ]:
epochs = 10

for epoch in range(epochs):
    for i, batch in enumerate(train_loader):
        data = batch[0]
        label = batch[1]
        
        with autograd.record():
            output = net(data)
            loss = loss_fn(output, label)
        
        loss.backward()
        trainer.step(batch_size)
        
        if i % 10 == 0:
            print(f'Epoch [{epoch}/{epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.mean().asscalar()}')

In [ ]:
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric

# Define the metric for evaluation
# VOC07MApMetric is commonly used for object detection tasks
metric = VOC07MApMetric(iou_thresh=0.5, class_names=classes)

# Iterate over the validation dataset
for batch in val_loader:
    data = batch[0]  # Images
    label = batch[1]  # Ground truth labels
    
    # Perform inference
    output = net(data)
    
    # Update the metric with predictions and ground truth
    metric.update(label, output)

# Compute and print the evaluation results
print("Evaluation Results:")
print(metric.get())

In [ ]:
from gluoncv.utils.viz import plot_bbox
import matplotlib.pyplot as plt

# Load a batch of validation data
for batch in val_loader:
    data = batch[0]  # Images
    label = batch[1]  # Ground truth labels
    
    # Perform inference
    output = net(data)
    
    # Get predictions
    bboxes, scores, cls_ids = output
    
    # Visualize the first image in the batch
    img = data[0].asnumpy().transpose((1, 2, 0))  # Convert to HWC format
    img = (img * 255).astype('uint8')  # Scale to 0-255 range
    
    # Plot ground truth bounding boxes
    gt_bboxes = label[0][:, :4]
    gt_cls_ids = label[0][:, 4:5]
    plot_bbox(img, gt_bboxes, scores=None, labels=gt_cls_ids, class_names=classes, colors=['green'] * len(gt_bboxes))
    
    # Plot predicted bounding boxes
    plot_bbox(img, bboxes[0], scores[0], cls_ids[0], class_names=classes, colors=['red'] * len(bboxes[0]))
    
    plt.show()
    break  # Only visualize the first batch